In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import yfinance as yf
from datetime import datetime, timedelta
from typing import List
from pydantic import BaseModel
from _watchlist import watchlist

In [106]:
ticker='NVDA'
date_offset=1

In [4]:
stock = yf.Ticker(ticker)

In [5]:
info = stock.info
company_name = info.get("shortName", ticker)

In [ ]:
# -----------------------------------------------------------
# 1. 설정 (종목과 날짜 지정)
# -----------------------------------------------------------
ticker_symbol = "NVDA"       # 종목 코드
target_date_str = "2026-01-05"  # 분석하고 싶은 날짜 (YYYY-MM-DD)
                               # 주의: 최근 60일 이내 평일이어야 데이터가 있습니다.

# -----------------------------------------------------------
# 2. 날짜 계산 (yfinance 쿼리용)
# -----------------------------------------------------------
target_date = datetime.strptime(target_date_str, "%Y-%m-%d")
next_day = target_date + timedelta(days=1) # 다음 날 (yfinance end 파라미터는 exclusive하므로)

# -----------------------------------------------------------
# 3. 전일 종가(Previous Close) 구하기
# -----------------------------------------------------------
# 타겟 날짜의 직전 거래일 종가를 알기 위해, 일주일 전부터 타겟 날짜까지의 일봉을 가져옵니다.
# end=target_date로 설정하면 타겟 날짜 당일은 포함되지 않으므로, 마지막 행이 '전일'이 됩니다.
ticker = yf.Ticker(ticker_symbol)
daily_df = ticker.history(start=target_date - timedelta(days=7), end=target_date, interval="1d")

if daily_df.empty:
    print("일별 데이터를 가져올 수 없습니다. 날짜를 확인해주세요.")
else:
    prev_close = daily_df['Close'].iloc[-1] # 전일 종가
    print(f"기준 전일 종가: {prev_close}")

    # -----------------------------------------------------------
    # 4. 당일 5분 봉 데이터 가져오기 & 변동률 계산
    # -----------------------------------------------------------
    # interval="5m"는 최대 60일 전까지만 조회 가능합니다.
    df_5m = ticker.history(start=target_date, end=next_day, interval="5m")

    if df_5m.empty:
        print("5분 봉 데이터가 없습니다. (휴장일이거나 60일 제한을 초과했을 수 있습니다.)")
    else:
        # 전일 종가 대비 변동률(%) 계산 공식: (현재가 - 전일종가) / 전일종가 * 100
        df_5m['Change_Pct'] = ((df_5m['Close'] - prev_close) / prev_close) * 100

        # 결과 확인 (시간, 종가, 변동률)
        print("\n--- 5분 봉 변동률 데이터 (상위 5개) ---")
        print(df_5m[['Close', 'Change_Pct']].head())

        print("\n--- 5분 봉 변동률 데이터 (하위 5개) ---")
        print(df_5m[['Close', 'Change_Pct']].tail())

기준 전일 종가: 188.85000610351562

--- 5분 봉 변동률 데이터 (상위 5개) ---
                                Close  Change_Pct
Datetime                                         
2026-01-05 09:30:00-05:00  192.975006    2.184273
2026-01-05 09:35:00-05:00  191.097000    1.189830
2026-01-05 09:40:00-05:00  190.229996    0.730733
2026-01-05 09:45:00-05:00  189.764206    0.484088
2026-01-05 09:50:00-05:00  190.559998    0.905476

--- 5분 봉 변동률 데이터 (하위 5개) ---
                                Close  Change_Pct
Datetime                                         
2026-01-05 15:35:00-05:00  187.654999   -0.632781
2026-01-05 15:40:00-05:00  187.794998   -0.558649
2026-01-05 15:45:00-05:00  188.104507   -0.394757
2026-01-05 15:50:00-05:00  187.770004   -0.571883
2026-01-05 15:55:00-05:00  188.110001   -0.391848


: 

In [117]:
date_offset = 2
target_date2 = datetime.now() - timedelta(days=date_offset-1)
print(f'{date_offset} days ago is {target_date2}')
stock.history(start=target_date2 - timedelta(days=7), end=target_date2, interval="1d")

2 days ago is 2026-01-07 01:12:38.625320


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-12-31 00:00:00-05:00,189.570007,190.559998,186.490005,186.500000,120100500,0.0,0.0
2026-01-02 00:00:00-05:00,189.839996,192.929993,188.259995,188.850006,148240500,0.0,0.0
2026-01-05 00:00:00-05:00,191.759995,193.630005,186.149994,188.119995,183529700,0.0,0.0
2026-01-06 00:00:00-05:00,190.520004,192.169998,186.820007,187.240005,176174700,0.0,0.0


In [118]:
target_date2

datetime.datetime(2026, 1, 7, 1, 12, 38, 625320)